In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2022-03-19 17:51:28--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.66.144, 142.250.207.80, 172.217.24.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.66.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  10.7MB/s    in 6.1s    

2022-03-19 17:51:35 (10.7 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
# unzip dataset from tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/media/victoru/B612CEC512CE8A37/ai50/pytorch_test/data/cat_and_dogs_filtered')
zip_ref.close()


In [3]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform

In [ ]:
#create dataset class for cats and dogs
class catsdogsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.label_dir = os.listdir(root_dir)
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = io.imread(img_name)
        label = self.label_dir
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample['image'] = self.transform(sample['image'])
            
        return sample

In [4]:
root_dir = '/media/victoru/B612CEC512CE8A37/ai50/pytorch_test/data/cat_and_dogs_filtered'
print(os.listdir(root_dir))

['cats_and_dogs_filtered']
